# Environment Implementation Turorial

In this tutorial we show how to implement your oun environment with your own problem. For this examples we have select a real life dataset with historical stock data from Apple company (https://www.kaggle.com/tarunpaparaju/apple-aapl-historical-stock-data) and we want to train a trading agent.

In [ ]:
from environments.env_base import EnvInterface, ActionSpaceInterface
from RL_Problem import rl_problem
from RL_Agent import ppo_agent_discrete_parallel
from RL_Agent.base.utils.networks import networks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten
from RL_Agent.base.utils import history_utils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import gc

Data from https://www.kaggle.com/tarunpaparaju/apple-aapl-historical-stock-data

En este tutorial vamos a crear un entorno para realizar inversiones en bolsa. Vamos a desarrollar el entorno sobre un
conjunto de datos reales y nuestro agente tendrá que decidir cuando conviene comprar y cuando conviene vender. Para
simplificar el problema para este ejemplo solo vamos a permitir al agente comprar una acción (que vamos a llamar stock
a partir de ahora para no confundirlas con las acciones del agente), por lo tanto, una vez ha comprado una unidad de
stock no podrá volver a comprar hasta que no decida vender la unidad de stock. Estamos ante un problema con acciones
discretas por lo que podemos usar un agente DQN y las acciones disponibles para el agente son: comprar (Buy), vender
(sell) y no hacer nada (idle), esta última puede referirse a esperar el momento idóneo para comprar o esperar el
momento idóneo para vender. El estado esta compuesto por una tupla (variación del precio stock, variación del
precio de stock comprado, flag stock en posesión). La variación del precio stock se refiere a la diferencia del precio
que había en el instante anterior y el actual; la variación del precio de stock comprado se refiere a la diferencia
entre el precio al que se adquirió stock y el precio actual, si no se dispone de stock porque todavía no se ha
comprado nada o porque se ha vendido esta característica tomará valor cero y por último, flag stock en posesión es una
característica binaria que si indica si tenemos stock en posesión, cuando tenemos stock vale 1 y en caso contrario 0.
Para que el agente tenga información temporal sobre como evoluciona el precio y de las decisiones que ha ido tomando
se apilan 20 estados para formar cada observación. Los datos se desarollan a lo largo de 10 años y para cada episodio
seleccionamos un dia de inicio aleatorio, el episodio se prolongará durante 100 iteraciones que se corresponderán con
los 100 días siguientes al seleccionados.
La funcion de recompensa premiará al agente cuando gane dinero por vender stock y le penalizará cuando lo pierda.
Por ello las acciones Buy y Idle se puentuan con reward = 0, la acción Sell se puntúa con la diferencia entre el
precio de compra del stock y el precio por el que se ha vendido. Cuando se llega a un estado final y el agente no ha
vendido todas sus acciones recive una penalización igual al dinero que perdería al venderlas en en el momento de
llegar al estado final, si en ese caso tueviese ganancias la recompensa será 0.

Definimos el espacio de acciones heredando de  environments.env_base.ActionSpaceInterface

## Trading Environment

The environment is an entity that implements a problem of interest on a reinforcement learning compatible way, this is as a decision making problem, more specifically, as Markov Decision Problem. 

Our particular problem have three posible actions: 1) buy stock, 2) sell stock and 3) ilde, do nothing The agent have to choose the best action with the objective of maximize the profit during a sequence of 100 days. Each day the agent can take an action taht is executet jut before the market closure time. To simplify the problem the agent only can have one unit of stock in each time step, this means that the agent has to sell the stock before buying again.

The state if formed by three elements: 

* Price variation: the variation of the stock prices between two consecutive days expresed in x times the last price. This is calculated as "price_variation = price(day)/price(day-1)" where "day" represent the current day.
* Gains: Potential profit if the stock is sold. If the agent do not have bought stock, "gains = 0". If the agent have bought stock, "gains = price(day)/price(buying_day)", where "day" represent the current day.
* Stock Bought: This is a flag that is set to "0." where the agent does not have stock in possesion and is set to "1." when the agent have stock in possesion.

The reward function return better values where the agent sell stock with good proffit. Where the agent sell stock losing money the reward will return worse values. Otherwise the reward will be neutral. Finally, if the episode terminate with stock in possesion the reward will be proportional to the money inverted to buy that stock.

Especifically, we use relatives values to the buy price and sell price "buy price/sell price" as reward, where the agent obtain profit "reward > 1" and if the agent obtain losses "reward < 1". In the other cases "reward = 1"

In the next cell we define the action space extending the "ActionSpaceInterface" from "environments.env_base.py". The action space requires tto set the number of actions "self.n". If the action space where continuous, we would be required to define the actions bounds in properties: 1) "self.low", low bound and 2) "self.high", high bound. Both bounds are float numbers and reference the overall maximun and minumum bounds for all possible actions.

In [ ]:
class action_space(ActionSpaceInterface):
    def __init__(self):
        """
        En un estorno con acciones discretas debemos definir el número de acciones self.n y un diccionario con las
        posibles acciones self.actions.
        """
        # Number of actions
        self.n = 3

        # buy -> buy stock, sell = sell stock y idle = do nothing.
        self.actions = {'buy': 0,
                        'sell': 1,
                        'idle': 2}

The next cell implements the environment itself extending from "EnvInterface" from "environments.env_base.py". 
We are required to define the next functions that complains with the OpenAI Gym interface:

* reset(): Reset the environment to an initial state. Receives nothing and Returns the state (ndarray).
* step(action): Execute an action in the environment producing a transition between the current state and the next state. Receives an action (ndarray) and Returns the next state (ndarray), reward (float), done (boolean. True if a terminal state is reached) and aditional iformation (dict). 
* render(): Render the environment in a graphical way or via command line. Receives nothing and Return nothing.
* close(): Close the rendering window.

In [ ]:
# Definimos el entorno
env = StockTrading()

In [ ]:
# Definimos la red neuronal de la forma mas avanzada que permite la libreria con un modelo secuencial de keras
def actor_lstm_custom_model(input_shape):
    actor_model = Sequential()
    actor_model.add(LSTM(128, input_shape=input_shape, activation='tanh'))
    actor_model.add(Dense(128, input_shape=input_shape, activation='relu'))
    actor_model.add(Dense(128, activation='relu'))

    return actor_model

def critic_lstm_custom_model(input_shape):
    critic_model = Sequential()
    critic_model.add(LSTM(128, input_shape=input_shape, activation='tanh'))
    critic_model.add(Dense(64, input_shape=input_shape, activation='relu'))
    critic_model.add(Dense(64, activation='relu'))

    return critic_model

In [ ]:
# Formateamos la red para pasarsela al aagente
net_architecture = networks.ppo_net(use_custom_network=True,
                                    actor_custom_network=actor_lstm_custom_model,
                                    critic_custom_network=critic_lstm_custom_model)

In [ ]:
class StockTrading(EnvInterface):
    """
    Environment for stock trading using historical stock data from Apple.
    https://www.kaggle.com/tarunpaparaju/apple-aapl-historical-stock-data
    """

    def __init__(self):
        super().__init__()

        # Define the actions and observation spaces. They are required properties.
        self.action_space = action_space()
        
        # The observation space is used by the library to know the state shape. In this case, we define a dummy 
        # (array of zeroes) because we do not need the values but we have acces to the shape.
        self.observation_space = np.zeros(3, )

        # Maximum iterations per episode.
        self.max_iter = 99

        # Load data
        dataset = pd.read_csv("/home/serch/TFM/CAPOIRL-TF2/tutorials/data/HistoricalQuotes.csv")

        # Invert data order to haver the correct cronological order.
        dataset = dataset.iloc[::-1]
        print(dataset.head())

        # Preprocess data
        self.data = dataset.iloc[:, 1].values
        self.data = [float(d.split()[0][1:]) for d in self.data]

        # Show dataset data
        fig, ax = plt.subplots(1)
        ax.plot(range(len(self.data)), self.data)
        ax.set(xlabel='days', ylabel='price $',
               title='Stock Trading Data')
        ax.grid()
        plt.show()


        # Auxiliar environment variables
        self.index_day = None  # Current day.
        self.profit = None  # Current profit
        self.stock_buying_price = 0. # Flag to know where the agent have stock in possesion.

        random.seed()
        
        # Buffer of days of buying and selling for rendering purposes.
        self.render_buy_index = []
        self.render_sell_index = []

    def reset(self):
        """
        Reset the environment to an initial state.
        :return: observation/state. numpy array of state shape
        """
        gc.collect()
        
        # Select a random init day.
        self.index_day = random.randint(11, len(self.data)-101)
        self.init_index = self.index_day

        price = self.data[self.index_day]

        # Create the inital state. [price_variation, gains, stock bought].
        state = np.array([1., 1., 0.])

        # Initialize control variables and buffers.
        self.last_action = self.action_space.actions['idle']
        self.last_state = state
        self.last_price = price
        self.first_price = price
        self.last_reward = 0.

        # Initializa auxiliar environment variables
        self.iter = 0
        self.profit = 0.
        self.index_day += 1
        self.stock_buying_price = 0.

        # Initialize rendering variables.
        self.render_buy_index = []
        self.render_sell_index = []

        return state

    def step(self, action):
        """
        Execute the action to get the next state.
        :param action: integer in [0, 3]
        :return: state:   numpy array of state shape.
                 reward: float
                 done: bool
                 info: dict or None
        """
        price = self.data[self.index_day]

        price_variation = price/self.last_price

        # Terminal state if the maximun number of iterations is reached.
        done = self.iter >= self.max_iter

        # Calculate reward.
        reward = 1.
        profit = 0.
        if action == self.action_space.actions['buy']:
            if self.stock_buying_price > 0:
                action = 4  # If we already have stock we can not buy any more. Action is "idle" buth we assing value 4 for rendering purposes.
            else:
                self.stock_buying_price = self.last_price  # Store the buying price.

        elif action == self.action_space.actions['sell']:
            if self.stock_buying_price > 0:
                profit = self.last_price - self.stock_buying_price  # Calculate profit.
                reward = self.last_price/self.stock_buying_price  # Calculate reward.

                self.stock_buying_price = 0.
            else:
                action = 3  # If we do not have stock we can not sell anything. Action is "idle" in this case, we assing value 3 for rendering purposes.
        
        # If the agent did not sell the stock onece the episode has finnished, we calculate the loss with respect to the buying price.
        if done and self.stock_buying_price > 0.:
            reward = self.last_price/self.stock_buying_price

        self.profit += profit

        gains = price/self.stock_buying_price if self.stock_buying_price > 0. else 0.


        # Create the inital state. [price_variation, gains, stock bought].
        state = np.array([price_variation, gains, 1. if self.stock_buying_price > 0. else 0.])


        self.last_state = state
        self.last_price = price
        self.last_action = action
        self.last_reward = reward
        self.iter += 1
        self.index_day += 1

        return state, reward, done, None

    def close(self):
        # Close the rendering figure
        plt.close(1)

    def render(self):
        plt.clf()

        fig = plt.figure(1)
        ax = fig.add_subplot(2, 1, 1)


        data = self.data

        # Get the current stock price.
        valor = data[self.index_day-2]
        ax.plot(range(100), data[self.init_index-1: self.init_index+99])
        ax.set(xlabel='days', ylabel='price $',
               title='Stock Trading')

        if self.last_action == self.action_space.actions['buy']:
            marker = "^"
            color = 'g'
            self.render_buy_index.append([self.iter, valor])
        elif self.last_action == self.action_space.actions['sell']:
            marker = "v"
            color = 'r'
            self.render_sell_index.append([self.iter, valor])
        elif self.last_action == 3:  # Sell, but without stock the action is transformed to idle.
            marker = "v"
            color = 'y'
        elif self.last_action == 4:  # Buy, but we already have stock so the action is tranformed to idle.
            marker = "^"
            color = 'y'
        else:  # 'idle'
            marker = "D"
            color = 'b'

        for s in self.render_sell_index:
            ax.plot(s[0], s[1], marker=7, color='r')

        for b in self.render_buy_index:
            ax.plot(b[0], b[1], marker=6, color='g')

        ax.plot(self.iter, valor, marker=marker, color=color)

        text1 = "profit: {:.1f}".format(self.profit)
        text2 = "   stock buying price: {:.1f}".format(self.stock_buying_price)
        text3 = "   current price: {:.1f}".format(valor)
        text4 = "   reward: {:.4f}".format(self.last_reward)
        ax.text(0.02, 0.95, text1 + text2 + text3 + text4, horizontalalignment='left', verticalalignment='center',
                transform=ax.transAxes)
        ax.grid()

        # We additionally show a sliding window of the last 20 days to see what the agent sees.
        ax2 = fig.add_subplot(2, 1, 2)
        ax2.plot(range(self.iter-19, self.iter+1), data[self.index_day-21: self.index_day - 1])
        ax2.set(xlabel='last 20 days', ylabel='price $',
               title='20 days window')
        ax2.plot(self.iter, valor, marker=marker, color=color)
        for s in self.render_sell_index:
            if self.iter - s[0] < 20:
                ax2.plot(s[0], s[1], marker="v", color='r')

        for b in self.render_buy_index:
            if self.iter - b[0] < 20:
                ax2.plot(b[0], b[1], marker="^", color='g')

        ax2.grid()

        plt.draw()
        plt.pause(0.01)


In [ ]:
agent = ppo_agent_discrete_parallel.Agent(actor_lr=1e-3,
                                         critic_lr=1e-3,
                                         batch_size=128,
                                         memory_size=100,
                                         epsilon=1.0,
                                         epsilon_decay=0.97,
                                         epsilon_min=0.15,
                                         net_architecture=net_architecture,
                                         n_stack=20,
                                         loss_critic_discount=0.001,
                                         loss_entropy_beta=0.01,
                                         tensorboard_dir='tensorboard_logs')

In [ ]:
from RL_Agent import dpg_agent

net_architecture = networks.dpg_net(use_custom_network=True,
                                    custom_network=actor_lstm_custom_model)
agent = dpg_agent.Agent(learning_rate=1e-4,
                        batch_size=64,
                        net_architecture=net_architecture,
                        n_stack=20,
                       tensorboard_dir='tensorboard_logs')

In [ ]:
problem = rl_problem.Problem(env, agent)

In [ ]:
problem.solve(1000, render=False)

In [ ]:
%matplotlib qt
# Probamos el rendimiento del agente
problem.test(render=True, n_iter=5)

In [ ]:
hist = problem.get_histogram_metrics()
history_utils.plot_reward_hist(hist, 10)

In [ ]:
!tensorboard --logdir=tensorboard_logs